In [11]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

print("Versions", tf.__version__)
print("Eager mode", tf.executing_eagerly())
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "Not Available")

Versions 2.15.0
Eager mode True
Hub version: 0.16.1
GPU is available


In [2]:
train_data, validation_data, test_data = tfds.load(name='imdb_reviews',split=('train[:60%]', 'train[60%:]','test'), as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.3GAC32_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.3GAC32_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.3GAC32_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [5]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423194 , -0.01190171,  0.06337537,  0.0686297 , -0.16776839,
        -0.10581177,  0.168653  , -0.04998823, -0.31148052,  0.07910344,
         0.15442258,  0.01488661,  0.03930155,  0.19772716, -0.12215477,
        -0.04120982, -0.27041087, -0.21922147,  0.26517656, -0.80739075,
         0.25833526, -0.31004202,  0.2868321 ,  0.19433866, -0.29036498,
         0.0386285 , -0.78444123, -0.04793238,  0.41102988, -0.36388886,
        -0.58034706,  0.30269453,  0.36308962, -0.15227163, -0.4439151 ,
         0.19462997,  0.19528405,  0.05666233,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201319 , -0.04418665, -0.08550781,
        -0.55847436, -0.2333639 , -0.20782956, -0.03543065, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862677,  0.7753425 , -0.07667087,
        -0.15752274,  0.01872334, -0.08169781, -0.3521876 ,  0.46373403,
        -0.08492758,  0.07166861, -0.00670818,  0.12686071, -0.19326551,
 

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x79968ee30160>>

In [7]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=10, validation_data=validation_data.batch(512), verbose=1)

Epoch 1/10
30/30 [==============================] - 10s 235ms/step - loss: 0.6820 - accuracy: 0.5027 - val_loss: 0.6380 - val_accuracy: 0.5243
Epoch 2/10
30/30 [==============================] - 6s 199ms/step - loss: 0.5886 - accuracy: 0.6157 - val_loss: 0.5501 - val_accuracy: 0.6965
Epoch 3/10
30/30 [==============================] - 7s 225ms/step - loss: 0.4771 - accuracy: 0.7671 - val_loss: 0.4527 - val_accuracy: 0.7938
Epoch 4/10
30/30 [==============================] - 7s 234ms/step - loss: 0.3601 - accuracy: 0.8544 - val_loss: 0.3787 - val_accuracy: 0.8357
Epoch 5/10
30/30 [==============================] - 6s 196ms/step - loss: 0.2702 - accuracy: 0.9010 - val_loss: 0.3376 - val_accuracy: 0.8540
Epoch 6/10
30/30 [==============================] - 7s 232ms/step - loss: 0.2052 - accuracy: 0.9321 - val_loss: 0.3171 - val_accuracy: 0.8642
Epoch 7/10
30/30 [==============================] - 5s 169ms/step - loss: 0.1565 - accuracy: 0.9525 - val_loss: 0.3075 - val_accuracy: 0.8686
Epoch

In [9]:
results = model.evaluate(test_data.batch(512), verbose=2)

49/49 - 2s - loss: 0.3443 - accuracy: 0.8563 - 2s/epoch - 32ms/step


In [10]:
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.344
accuracy: 0.856


In [96]:
sample_text = ("The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.")
predictions = model.predict(np.array([sample_text]))

1/1 [==============================] - 0s 100ms/step


In [97]:
predictions[0]

array([1.2169622], dtype=float32)

In [99]:
def translate(prediction):
    threshold = 0.5
    label = 'positive' if predictions[0] >= threshold else 'negative'
    print(f'The review is: {label}')


In [100]:
translate(predictions[0])

The review is: positive


In [101]:
sample_text2 = ("The movie was so bad, that i had fall asleep in the middle of it, if only there were some action to the movie.")
predictions = model.predict(np.array([sample_text2]))

1/1 [==============================] - 0s 19ms/step


In [102]:
predictions[0]

array([-3.9070826], dtype=float32)

In [103]:
translate(predictions[0])

The review is: negative
